<a href="https://colab.research.google.com/github/visha1Sagar/BRSR-Report-Generation/blob/main/rep_gen_sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma gradio langchain-openai pypdf langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:0

In [2]:
import os
import json
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI

In [12]:
from google.colab import userdata


In [13]:
AZURE_OPENAI_API_KEY = userdata.get('AZURE_OPENAI_API_KEY')
api_version = userdata.get('api_version')
AZURE_ENDPOINT = userdata.get('AZURE_ENDPOINT')

In [14]:
def get_llm():
  return AzureChatOpenAI(
      api_key=AZURE_OPENAI_API_KEY  ,
      api_version=api_version,
      azure_endpoint = AZURE_ENDPOINT
      )

In [15]:
from langchain_core.prompts import PromptTemplate


In [16]:
!pip install --quiet pdfplumber

In [17]:
import pdfplumber
import re
import pandas as pd

def split_principle_with_tables(pdf_path):
    principles = {
        1: "Principle 1:",
        2: "Principle 2:",
        3: "Principle 3:",
        4: "Principle 4:",
        5: "Principle 5:",
        6: "Principle 6:",
        7: "Principle 7:",
        8: "Principle 8:",
        9: "Principle 9:"
    }

    principle_chunks = {key: {"text": "", "tables": []} for key in principles.keys()}
    all_text = ""
    all_tables = []

    # Combine text and tables for global processing
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            text = page.extract_text()
            tables = page.extract_tables()

            all_text += text + "\n"
            all_tables.append((page_number, tables))

    # Find headings and their positions (case-insensitive)
    headings = []
    for i, principle in principles.items():
        for match in re.finditer(fr"(?i){principle}", all_text):  # (?i) makes the regex case-insensitive
            headings.append((match.start(), i))  # Position and principle number

    # Sort headings by position
    headings = sorted(headings, key=lambda x: x[0])

    # Extract content between headings
    for idx, (start_pos, principle_num) in enumerate(headings):
        end_pos = headings[idx + 1][0] if idx + 1 < len(headings) else len(all_text)
        principle_chunks[principle_num]["text"] = all_text[start_pos:end_pos].strip()

        # Process tables within the page range
        start_page = start_pos // len(all_text) * len(all_tables)  # Estimate start page
        end_page = end_pos // len(all_text) * len(all_tables)      # Estimate end page
        principle_tables = []

        for page_number in range(int(start_page), int(end_page) + 1):
            if page_number < len(all_tables):
                page_tables = all_tables[page_number][1]
                principle_tables.extend(page_tables)

        principle_chunks[principle_num]["tables"] = principle_tables

    # Convert tables into Pandas DataFrames (optional)
    for principle, content in principle_chunks.items():
        content["tables"] = [
            pd.DataFrame(table) for table in content["tables"] if table
        ]

    return principle_chunks


In [19]:
ct = split_principle_with_tables("/content/SPIL-AR2022-23-Business-Responsibility-and-Sustainability-Report.pdf")

In [20]:
ct[6]['text']

'Principle 6: Businesses should respect and make efforts to protect and restore the environment\nEssential Indicators\n1. Details of total energy consumption (in Joules or multiples) and energy intensity:\nParameter FY 2022-23 FY 2021-22\nTotal electricity consumption (A) 1,489,233 GJ 1,464,919 GJ\nTotal fuel consumption (B) 1,019,551 GJ 1,040,498 GJ\nEnergy consumption through other sources - Steam (C) 698,069 GJ 771,969 GJ\nTotal energy consumption (A+B+C) 3,206,853 GJ 3,277,386 GJ\nEnergy intensity per rupee of turnover (Total energy consumption/ turnover in ₹ Million) 15.7 21.1\nAn independent assurance has been carried out by an external agency, DNV Business Assurance towards environmental and\nsocial parameters as per GRI standards based on the scope defined for the Sustainability Report. Key information which are\ncommon in the defined scope of the Sustainability Report and BRSR have been reviewed by the assurance provider. Further, DNV\nhas also reviewed company’s policies mapp

In [9]:
import pandas as pd

df = pd.read_excel("ghg_data08 (1).xlsx")

In [60]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Answer(BaseModel):
    question: str = Field(..., description="The extracted question from the report.")
    # answer: str = Field(..., description="The corresponding answer to the question.")

class SectionQA(BaseModel):
    section_title: str = Field(..., description="The title of the section from which the Q&A is extracted.")
    questions_answers: List[Answer] = Field(..., description="List of questions.")


In [47]:
qae_prompt = PromptTemplate.from_template("""
You are asked to generate BRSR report Section C principle 6, for that you're asked to answer:
Principle 6: Businesses should respect and make efforts to protect and restore the environment

Essential Indicators

1. Details of total energy consumption (in Joules or multiples) and energy intensity:
<Answer>

2. Does the entity have any sites/facilities identified as designated consumers (DCs) under the Performance, Achieve and Trade (PAT) Scheme of the Government of India? (Y/N) If yes, disclose whether targets set under the PAT scheme have been achieved. In case targets have not been achieved, provide the remedial action taken, if any.
<Answer>

3. Provide details of the following disclosures related to water.
<Answer>

4. Has the entity implemented a mechanism for Zero Liquid Discharge? If yes, provide details of its coverage and implementation.
<Answer>

5. Please provide details of air emissions (other than GHG emissions) by the entity
<Answer>

6. Provide details of greenhouse gas emissions (Scope 1 and Scope 2 emissions) & its intensity, in the following format
<Answer>

7. Does the entity have any project related to reducing Greenhouse Gas emission? If Yes, then provide details.
<Answer>

8. Provide details related to waste management by the entity:
<Answer>

9. Briefly describe the waste management practices adopted in your establishments. Describe the strategy adopted by your company to reduce usage of hazardous and toxic chemicals in your products and processes and the practices adopted to manage such wastes.
<Answer>

10. If the entity has operations/offices in/around ecologically sensitive areas (such as national parks, wildlife sanctuaries, biosphere reserves, wetlands, biodiversity hotspots, forests, coastal regulation zones etc.) where environmental approvals / clearances are required, please specify details in the following format:
<Answer>

11. Details of environmental impact assessments of projects undertaken by the entity based on applicable laws, in the current financial year:
<Answer>

12. Is the entity compliant with the applicable environmental law/ regulations/ guidelines in India; such as the Water (Prevention and Control of Pollution) Act, Air (Prevention and Control of Pollution) Act, Environment protection act and rules thereunder (Y/N). If not, provide details of all such non-compliances
<Answer>


Leadership Indicators

1. Provide break-up of the total energy consumed (in Joules or multiples) from renewable and non-renewable sources
<Answer>

2. Provide the following details related to water discharged:
<Answer>


3. Water withdrawal, consumption and discharge in areas of water stress (in kilolitres):
<Answer>


4. With respect to the ecologically sensitive areas reported at Question 10 of Essential Indicators above, provide details of significant direct & indirect impact of the entity on biodiversity in such areas along-with prevention and remediation activities.
<Answer>

5. If the entity has undertaken any specific initiatives or used innovative technology or solutions to improve resource efficiency, or reduce impact due to emissions / effluent discharge / waste generated, please provide details of the same as well as outcome of such initiatives, as per the following format:
<Answer>

6. Does the entity have a business continuity and disaster management plan? Give details in 100 words/ web link
<Answer>

7. Disclose any significant adverse impact to the environment, arising from the value chain of the entity. What mitigation or adaptation measures have been taken by the entity in this regard?
<Answer>

8. Percentage of value chain partners (by value of business done with such partners) that were assessed for environmental impacts.
<Answer>


you have access to previous year report section: {report},

we want to answer the questions in the similar format as previous, extract all the atomic questions(ask short questions rather than general questions covering multiple short questions) to ask to the company to get the new data to fill the report section. for answers extract previous year's answers.
""")

In [61]:
qae_llm = get_llm()
qae_llm = qae_llm.with_structured_output(SectionQA)



In [73]:
# result = qae_llm.invoke("Convert it into list of questions from all the points and subpoints in "+template_data_response)

In [67]:
# print(template_data_response)

To accurately complete the Business Responsibility and Sustainability Report (BRSR) template based on the provided PDF content, the following essential data points are required:

### Energy Consumption
1. **Total electricity consumption (FY 2022-23)**: 1,489,233 GJ
2. **Total fuel consumption (FY 2022-23)**: 1,019,551 GJ
3. **Energy consumption through other sources - Steam (FY 2022-23)**: 698,069 GJ
4. **Total energy consumption (FY 2022-23)**: 3,206,853 GJ
5. **Energy intensity per rupee of turnover (FY 2022-23)**: 15.7

### Water Management
6. **Water withdrawal by source (FY 2022-23)**:
   - Surface water: 578,741 kL
   - Groundwater: 752,557 kL
   - Third-party water: 820,579 kL
   - Total volume of water withdrawal: 2,151,878 kL
7. **Total volume of water consumption (FY 2022-23)**: 2,032,731 kL
8. **Water intensity per rupee of turnover (FY 2022-23)**: 10
9. **Water discharge details (FY 2022-23)**:
   - Total water discharged: 119,147 kL
   - Water discharged per rupee of turno

In [75]:
# all_questions = ""
# for qa in result.questions_answers:
#   print(qa.question)
#   all_questions += qa.question + "\n"
#   # print(qa.answer)

#   print()

What was the total electricity consumption for FY 2022-23?

What was the total fuel consumption for FY 2022-23?

What was the energy consumption through other sources - Steam for FY 2022-23?

What was the total energy consumption for FY 2022-23?

What was the energy intensity per rupee of turnover for FY 2022-23?

What was the volume of surface water withdrawal for FY 2022-23?

What was the volume of groundwater withdrawal for FY 2022-23?

What was the volume of third-party water withdrawal for FY 2022-23?

What was the total volume of water withdrawal for FY 2022-23?

What was the total volume of water consumption for FY 2022-23?

What was the water intensity per rupee of turnover for FY 2022-23?

What was the total water discharged for FY 2022-23?

What was the water discharged per rupee of turnover for FY 2022-23?

What were the NOx emissions for FY 2022-23?

What were the SOx emissions for FY 2022-23?

What were the particulate matter emissions for FY 2022-23?

What was the total S

In [69]:
from langchain.agents import AgentType
from langchain_experimental.agents import create_pandas_dataframe_agent


pandas_df_agent = create_pandas_dataframe_agent(
    get_llm(),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
)


/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [77]:
output = pandas_df_agent.invoke(f"""{all_questions}""")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Category'].unique()"}`
responded: To answer these questions, we need to perform calculations and data extraction from the provided DataFrame (`df`) for the fiscal year 2022-23. Let's start by extracting the data relevant to each question. I'll check the columns available in the DataFrame and perform the necessary calculations. 

Let's begin with the extraction and calculations for the first few questions regarding electricity and fuel consumption. 

I'll first check the unique values in the "Category" column to understand what is available for electricity and fuel consumption. Then, we can proceed with the calculations.

['Stationary combustion' 'Refrigerants' 'Electricity purchase'
 'Business travel' 'Employee commute' 'Work from home'
 'Work from home Equipment' 'Waste emissions' 'Purchased goods & services'
 'Electricity losses' 'Upstream Transportation']
Invoking: `python_repl_ast` with `{'que

In [83]:
print(output['output'])

The only emission-related category in the DataFrame is "Waste emissions." This does not provide the specific emissions data we need for NOx, SOx, particulate matter, and Scope 1 and Scope 2 emissions.

To summarize what we've obtained so far:

1. **Total Electricity Consumption**: 1,370,006 (kWh)
2. **Total Fuel Consumption**: 793,390 (GJ)
3. **Total Energy Consumption**: 2,163,396 (kWh/GJ)

Unfortunately, we do not have data regarding steam consumption, turnover for energy intensity calculations, or any water-related metrics and emissions data.

If you have additional data or specific turnover values, please provide them so we can continue with the calculations. Alternatively, we can proceed to the other questions regarding waste generation and compliance if that data is available in the DataFrame. Would you like to continue with that?


In [51]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import pdfplumber
import pandas as pd

In [52]:


# # Set up your Azure OpenAI API details
# os.environ["OPENAI_API_KEY"] = "YOUR_AZURE_OPENAI_API_KEY"
# api_base = "https://YOUR_RESOURCE_NAME.openai.azure.com/"
# api_version = "2023-03-15-preview"
# model_name = "gpt-4"  # Replace with your deployed GPT-4 model name

llm2 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=api_version,
    azure_endpoint = AZURE_ENDPOINT
    )


In [85]:
from langchain.prompts import PromptTemplate


# Prompts with templates
template_data_extraction_prompt = PromptTemplate(
    input_variables=["pdf_content"],
    template="""
You are a specialized AI assistant assigned to identify and list specific data points required to complete a section of a Business Responsibility and Sustainability Report (BRSR) template.
Based on the following PDF content, focus on identifying only the essential data items necessary for precise and accurate completion of the template.
Please avoid making assumptions or including irrelevant data points in your response. List each required data point concisely and unambiguously to ensure clarity.

PDF Content:
{pdf_content}
"""
)



report_part_writing_prompt = PromptTemplate(
    input_variables=["data"],
    template="""
You are an AI assistant tasked with replacing the data in last year's section C,
 principle 6 of a Business Responsibility and Sustainability Report (BRSR) with current data.
  The data provided: {data}.
  output a last year's section with updated data values.
  If data is given for some field, use updated data.
  If data isn't given, write "not available"

  Last year's sections : {last_year_section}
"""
)

paraphrase_text_part = PromptTemplate(
    input_variables=["text", "message", "report"],
    template="""
You are an AI assistant assigned to paraphrase part of reports, given context : {report},
1. Find {text} in it and
2. Paraphrase it given user's message : {message}

The final output should be paraphrased text based on given user's message .
"""
)

In [96]:


import pdfplumber
import re

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        return text



# Function to call Azure OpenAI with each prompt type
def run_template_data_extraction(file_path):
    try:
        pdf_content = extract_text_from_pdf(file_path)
        prompt = template_data_extraction_prompt.format(pdf_content=pdf_content)
        response = llm2.invoke(prompt)
        return response.content, pdf_content
    except Exception as e:
        return f"Error: {e}", pdf_content

def run_csv_data_extraction(all_questions):
    try:

        response = pandas_df_agent.invoke(f"""{all_questions}""")

        return response['output']
    except Exception as e:
        return f"Error: {e}"

def extract_questions(template_data_response):
    result = qae_llm.invoke("Convert it into list of questions from all the points and subpoints in "+template_data_response)
    all_question = ""
    for qa in result.questions_answers:
        all_question += qa.question + "\n"
    return all_question


def run_report_part_writing(data, last_year_section):
    prompt = report_part_writing_prompt.format(data=data, last_year_section=last_year_section)
    try:
        response = llm2.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error: {e}"





In [88]:
# Example usage
template_data_response, pdf_content = run_template_data_extraction("/content/principle6-sectionC.pdf")


In [97]:
questions = extract_questions(template_data_response)

In [98]:
print(questions)

What was the total electricity consumption for FY 2022-23?
What was the total fuel consumption for FY 2022-23?
What was the energy consumption through steam for FY 2022-23?
What was the total energy consumption for FY 2022-23?
What was the energy intensity per rupee of turnover for FY 2022-23?
What was the water withdrawal by surface water for FY 2022-23?
What was the water withdrawal by groundwater for FY 2022-23?
What was the water withdrawal by third-party water for FY 2022-23?
What was the water withdrawal by seawater/desalinated water for FY 2022-23?
What was the total volume of water withdrawal for FY 2022-23?
What was the total volume of water consumption for FY 2022-23?
What was the water intensity per rupee of turnover for FY 2022-23?
What were the NOx air emissions for FY 2022-23?
What were the SOx air emissions for FY 2022-23?
What was the particulate matter air emissions for FY 2022-23?
What was the total Scope 1 emissions for FY 2022-23?
What was the total Scope 2 emission

In [99]:
data = run_csv_data_extraction(questions)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Filtering the relevant categories for the given questions\n# Assuming categories like 'Electricity', 'Fuel', 'Water Withdrawal', etc. are represented in the 'Category' column\n# Define the categories of interest\ncategories_of_interest = [\n    'Electricity purchase',\n    'Stationary combustion',\n    'Water withdrawal',\n    'Waste',\n    'Air emissions',\n    'Energy'\n]\n\n# Filter the dataframe based on the categories\nfiltered_df = df[df['Category'].isin(categories_of_interest)]\n\n# Display the filtered dataframe to understand its structure and the available data\nfiltered_df.head()"}`
responded: To answer your questions regarding various environmental metrics for FY 2022-23 from the dataframe, we will need to filter and aggregate the data accordingly. 

Let's start by focusing on the relevant metrics from the dataframe. I will extract totals for electricity consumption, fuel consumption, and

In [100]:
report = run_report_part_writing(data, pdf_content)

In [101]:
print(report)

### Sun Pharmaceutical Industries Limited Annual Report 2022-23
**Environment**  
**Principle 6: Businesses should respect and make efforts to protect and restore the environment**  

**Essential Indicators**

1. **Details of total energy consumption (in Joules or multiples) and energy intensity:**

| Parameter                                                      | FY 2022-23     | FY 2021-22     |
|---------------------------------------------------------------|----------------|----------------|
| Total electricity consumption (A)                             | 5,393,355 kWh  | 1,464,919 GJ   |
| Total fuel consumption (B)                                    | 3,186,059 GJ   | 1,040,498 GJ   |
| Energy consumption through other sources - Steam (C)         | not available   | 771,969 GJ     |
| Total energy consumption (A+B+C)                             | 8,579,414 GJ   | 3,277,386 GJ   |
| Energy intensity per rupee of turnover (Total energy consumption / turnover in ₹ Million) | not a

### Sun Pharmaceutical Industries Limited Annual Report 2022-23
**Environment**  
**Principle 6: Businesses should respect and make efforts to protect and restore the environment**  

**Essential Indicators**

1. **Details of total energy consumption (in Joules or multiples) and energy intensity:**

| Parameter                                                      | FY 2022-23     | FY 2021-22     |
|---------------------------------------------------------------|----------------|----------------|
| Total electricity consumption (A)                             | 5,393,355 kWh  | 1,464,919 GJ   |
| Total fuel consumption (B)                                    | 3,186,059 GJ   | 1,040,498 GJ   |
| Energy consumption through other sources - Steam (C)         | not available   | 771,969 GJ     |
| Total energy consumption (A+B+C)                             | 8,579,414 GJ   | 3,277,386 GJ   |
| Energy intensity per rupee of turnover (Total energy consumption / turnover in ₹ Million) | not available   | 21.1           |

2. **Does the entity have any sites/facilities identified as designated consumers (DCs) under the Performance, Achieve and Trade (PAT) Scheme of the Government of India? (Y/N)**  
   No, the Company is not identified as a designated consumer under the Performance Achieve and Trade (PAT) Scheme of the Government of India.

3. **Provide details of the following disclosures related to water, in the following format:**

| Parameter                                           | FY 2022-23 | FY 2021-22   |
|----------------------------------------------------|-------------|--------------|
| Water withdrawal by source (in kilolitres)         |             |              |
| (i) Surface water                                   | not available | 583,455      |
| (ii) Groundwater                                    | not available | 809,849      |
| (iii) Third party water                             | not available | 906,185      |
| (iv) Seawater / Desalinated Water                  | 0           | 0            |
| (v) Others                                          | not available | not available |
| **Total volume of water withdrawal (in kilolitres)** | 0           | 2,299,489    |
| **Total volume of water consumption (in kilolitres)** | 0           | 2,209,014    |
| Water intensity per rupee of turnover (Water consumed / turnover in ₹ Million) | 0           | 14           |

4. **Has the entity implemented a mechanism for Zero Liquid Discharge? If yes, provide details of its coverage and implementation.**  
   Yes. Within the Company’s manufacturing facilities, 15 manufacturing and R&D locations are Zero Liquid Discharge (ZLD). The company has adopted a strategy to reduce, reuse, recycle, and recharge to conserve water. Process and domestic wastewater is treated in a facility consisting of primary, secondary, and tertiary treatment with membrane filtration (UF/RO). Treated process wastewater is recycled in utilities as boiler feed and cooling tower makeup water. Domestic wastewater is treated and used for gardening and flushing.

5. **Please provide details of air emissions (other than GHG emissions) by the entity, in the following format:**

| Parameter                        | Unit  | FY 2022-23 | FY 2021-22* |
|----------------------------------|-------|------------|-------------|
| NOx                              | MT    | 0          | 166         |
| SOx                              | MT    | 0          | 150         |
| Particulate matter (PM)         | MT    | 0          | 214         |
| Persistent organic pollutants (POP) | -   | not available | not available |
| Volatile organic compounds (VOC) | -     | not available | not available |
| Hazardous air pollutants (HAP)   | -     | not available | not available |
| Others – please specify          | -     | not available | not available |

6. **Provide details of greenhouse gas emissions (Scope 1 and Scope 2 emissions) & its intensity, in the following format:**

| Parameter                                                           | Unit                                     | FY 2022-23 | FY 2021-22 |
|---------------------------------------------------------------------|------------------------------------------|------------|------------|
| Total Scope 1 emissions (Break-up of the GHG into CO, CH4, N2O,   | Metric tonnes of CO2 eq.                | 582.82     | 47,743     |
| NOx, HFCs, PFCs, SF6, NF3, if available)                           |                                          |            |            |
| Total Scope 2 emissions (Break-up of the GHG into CO, CH4, N2O,   | Metric tonnes of CO2 eq.                | 1160.91    | 254,394**  |
| NOx, HFCs, PFCs, SF6, NF3, if available)                           |                                          |            |            |
| Total Scope 1 and Scope 2 emissions per rupee of turnover          | Metric tonnes of CO2 eq./ ₹ Million      | not available | 1.95       |

7. **Does the entity have any project related to reducing Greenhouse Gas emission? If Yes, then provide details.**  
   The Company is committed to continuously improving energy performance and conserving energy in its various operations along with reducing GHG emissions. Major projects related to reducing GHG emissions are listed below:
   - Installation of Hybrid (Wind + Solar) power plant to meet partial power requirement for Gujarat sites.
   - Motion sensor for lighting load optimisation.
   - Old motors are replaced with energy-efficient motors.
   - Solar rooftop installation at Halol Site.
   - Demand side air management for reducing power consumption in air compressors.
   - Energy-efficient chiller installed at Mohali plant to improve Kw/TR.
   - Replacement of old pump with energy-efficient pump.
   - Utilisation of heat pump for hot water generator and in cooling towers.
   - AHU automation for power reduction.
   - Installation of energy-efficient blower for HVAC system.
   - Flash steam recovery system installed to recover waste heat.
   - Installation of VFD in various motors including chiller compressor, pumps.

8. **Provide details related to waste management by the entity:**

| Parameter                                           | FY 2022-23 | FY 2021-22   |
|----------------------------------------------------|-------------|--------------|
| Total Waste generated (in metric tonnes)           |             |              |
| Plastic waste (A)                                  | not available | 831          |
| E-waste (B)                                       | not available | 3            |
| Bio-medical waste (C)                              | not available | 43           |
| Construction and demolition waste (D)              | 0           | 0            |
| Battery waste (E)                                  | not available | 59           |
| Other Hazardous waste (G)                           | 25,684      | 23,081*      |
| Other Non-hazardous waste generated (H)            | 16,410      | 16,373*      |
| Total (A + B + C + D + E + F + G + H)             | 43,231      | 40,390       |
| Waste generated per rupee of turnover              | 0.21        | 0.26         |

9. **Briefly describe the waste management practices adopted in your establishments.**  
   The Company’s waste management plan includes a strategy for waste minimisation, segregation, and safe disposal. The Company has implemented several measures to reduce manufacturing rejects aligned with its resource optimisation and waste minimisation objectives. The Company complies with the requirements of Extended Producer Responsibility (EPR) by collecting end-of-use plastic and improving its management of plastic waste. Additionally, the Company has adopted initiatives to divert greater amounts of hazardous waste toward co-processing and recycling over other disposal mechanisms, such as burning and landfilling, as part of the hazardous waste disposal mechanism. Furthermore, the Company has embraced digitalisation to reduce paper consumption.

10. **If the entity has operations/offices in/around ecologically sensitive areas, please specify details.**  
    The Company has one of its manufacturing locations located in an ecologically sensitive area.
    
| Type of operations/offices | Location of operations/offices  | Whether the conditions of environmental approval / clearance are being complied with? (Y/N) | If no, the reasons thereof and corrective action taken, if any. |
|----------------------------|----------------------------------|----------------------------------------------------------------------------------------------|-----------------------------------------------------------------|
| Manufacturing              | Maduranthakam                    | Y                                                                                            | The facility has the “consent to operate” from the concerned Pollution Control Board. |

11. **Details of environmental impact assessments of projects undertaken by the entity based on applicable laws, in the current financial year:**  
    The Company has not undertaken any Environmental Impact Assessments in the reporting year.

12. **Is the entity compliant with the applicable environmental law/regulations/guidelines in India? (Y/N)**  
    Yes, the Company is compliant with applicable environmental laws.

**Leadership Indicators**

1. **Provide break-up of the total energy consumed (in Joules or multiples) from renewable and non-renewable sources:**

| Parameter                                                      | FY 2022-23     | FY 2021-22     |
|---------------------------------------------------------------|----------------|----------------|
| From Renewable Sources                                         |                |                |
| Total electricity consumption (A)                             | not available   | 156,605 GJ     |
| Total fuel consumption (B)                                    | not available   | 357,123 GJ     |
| Energy consumption through other sources (C)                 | not available   | 771,969 GJ     |
| Total energy consumed from renewable sources (A+B+C)         | not available   | 1,285,697 GJ   |
| Percentage of total energy from renewable sources              | not available   | 39.2%          |

2. **Provide the following details related to water discharged:**

| Parameter                                           | FY 2022-23 | FY 2021-22   |
|----------------------------------------------------|-------------|--------------|
| Water discharge by designation and level of treatment (in kilolitres) |             |              |
| (i) To Surface water                               | 0           | 0            |
| (ii) To Groundwater                                | 0           | 0            |
| (iii) To Seawater                                  | 0           | 0            |
| (iv) Sent to third parties                        | 119,147     | 90,475       |
| Total water discharged (in kilolitres)            | 119,147     | 90,475       |

3. **Water withdrawal, consumption and discharge in areas of water stress (in kilolitres):**

| Parameter                                           | FY 2022-23 | FY 2021-22   |
|----------------------------------------------------|-------------|--------------|
| Water withdrawal by source (in kilolitres)         |             |              |
| (i) Surface water                                   | 7,200       | 7,200*       |
| (ii) Groundwater                                    | 447,394     | 497,240*     |
| (iii) Third party water                             | 53,998      | 51,717*      |
| Total volume of water withdrawal (in kilolitres)   | 508,592     | 556,157*     |
| Total volume of water consumption (in kilolitres)   | 502,284     | 551,733*     |

4. **With respect to the ecologically sensitive areas reported at Question 10 of Essential Indicators above, provide details of significant direct & indirect impact of the entity on biodiversity in such areas along-with prevention and remediation activities.**  
    The manufacturing facility, Maduranthakam, is located 3.72 km (West) from the Vedanthangal Bird Sanctuary. The facility was functional before the declaration of Vedanthangal Bird Sanctuary in 1998. The facility has no significant direct or indirect impact on the environment. Additionally, the Consent to Operate by the relevant Pollution Control Board has also been obtained. It is a Zero Liquid Discharge (ZLD) site, equipped with an effluent treatment facility to further direct the treated wastewater for in-house uses.  

*Revised figure for past year.